# Gaussian Elimination

Gaussian elimination is detailed in Sec. 6.1.1. The process is iterative to the point of being tedious and lends itself nicely to a nested for loop. 

There is some example code given, but as usual it is instructive to attempt the exercise for yourself first. Write some code to perform Gaussian Elimination for the example Eq. 6.2.

Once you have the resulting upper triangular, write another loop to do the back substitution and solve the system of equations. The correct answer is given in the text.

In [2]:
# First we need to actually get that array in our code so:
import numpy as np

A = np.array([[2,1,4,1],
            [3,4,-1,-1],
            [1,-4,1,5],
            [2,-2,1,3]],float)

x = np.array([-4,3,9,7],float)


# Process: Take A11 and divide the first row by it. Then subtract the 3 following rows by this new first row times the first value in the rows
# Hey so first question was how do I find a specific value? Do this notation [i,j] where i,j is the indexes for the specific value.
#print(A[0,1])
#print(A[0])
#print(A[0,1])

for i in range(0,len(A)):
    x[i] = x[i] / A[i,i]
    A[i,:] = A[i,:] / A[i,i] # This makes Diagonal entries one
    for j in range(i+1,len(A)): # i+1 makes it so no dividing by zero, always appreciated
        x[j] = x[j] - (x[i] * A[j,i])
        A[j,:] = A[j,:] - (A[i,:] * A[j,i]) # Subtraction for upper triangular
    print(A)


answer = np.empty(len(x),float)


#Generalize this later:
x4 = x[3]
x3 = x[2] - A[2,3]*x4
x2 = x[1] - A[1,2]*x3 - A[1,3]*x4
x1 = x[0] - A[0,1]*x2 - A[0,2]*x3 - A[0,3]*x4

answer = [x1,x2,x3,x4]
print(answer)

[[ 1.   0.5  2.   0.5]
 [ 0.   2.5 -7.  -2.5]
 [ 0.  -4.5 -1.   4.5]
 [ 0.  -3.  -3.   2. ]]
[[  1.    0.5   2.    0.5]
 [  0.    1.   -2.8  -1. ]
 [  0.    0.  -13.6   0. ]
 [  0.    0.  -11.4  -1. ]]
[[ 1.   0.5  2.   0.5]
 [ 0.   1.  -2.8 -1. ]
 [-0.  -0.   1.  -0. ]
 [ 0.   0.   0.  -1. ]]
[[ 1.   0.5  2.   0.5]
 [ 0.   1.  -2.8 -1. ]
 [-0.  -0.   1.  -0. ]
 [-0.  -0.  -0.   1. ]]
[2.0000000000000036, -1.000000000000003, -2.0, 0.9999999999999964]


# LU Decomposition

LU Decomposition is similar to Gaussian elimination, but is more flexible. The result is a set of matrices LU that are equivalent to A, but can be used to immediately solve any problem of the form $ \mathbf{A} \vec{v} = \vec{x_{n}}$ where we might have many different $\vec{x_{n}}$ for which we need $\vec{v}$.

The suggestion in the text is to use an iterative process to build up the matrices L and U.

Begin by setting up the problem

In [2]:
import numpy as np

A = np.array([[2, 1, 4, 1],
             [3, 4, -1, -1],
             [1, -4, 1, 5],
             [2, -2, 1, 3]], float)
v =  [-4, 3, 9, 7]
N = A.shape[0]

L = np.zeros([N,N], float)
U = np.copy(A)

Now we'll iterate through the rows and columns. Notice I've created U to initially be a copy of **A** while **L** is zeros. 

In the text, we write a series of matrices whose product is the upper triangular Gaussian elimination product. In practice, we can build this product iteratively.

The first column of the matrix **L** is the first row of **A**, $a_{00}$ to $a_{n0}$. The first row in **U** is the Gaussian elimination row, $a_{00}/a_{00}$ through $a_{n0}/a_{00}$.

Then we're done editing that row in **U**, but we need to subract it multiplied by the first element of the remaining rows from the remaining rows. Now we have second matrix with elements $b_{nm}$.

The process is iterative from here. We select the next column of **L** from b, then normalize b and use the result to calculate c.

This process is of course a nested for loop. Try to code it. It may be helpful to print L and U at each step.

Verify your result by subtracting the product **LU** from **A** e.g., `print(np.matmul(L,U) - A)`

In [3]:
A = np.array([[2, 1, 4, 1],
             [3, 4, -1, -1],
             [1, -4, 1, 5],
             [2, -2, 1, 3]], float)
v =  [-4, 3, 9, 7]
N = A.shape[0]

L = np.zeros([N,N], float)
U = np.copy(A)

# code
for i in range(0,N):
    L[i:N,i] = U[i:N,i]
    print(L)
    U[i,:] = U[i,:] / U[i,i]
    for j in range(i+1,N):
        mul = U[j,i]
        U[j,:] -=  (mul * U[i,:])
    # print(U)
        
print(np.matmul(L,U) - A)

[[2. 0. 0. 0.]
 [3. 0. 0. 0.]
 [1. 0. 0. 0.]
 [2. 0. 0. 0.]]
[[ 2.   0.   0.   0. ]
 [ 3.   2.5  0.   0. ]
 [ 1.  -4.5  0.   0. ]
 [ 2.  -3.   0.   0. ]]
[[  2.    0.    0.    0. ]
 [  3.    2.5   0.    0. ]
 [  1.   -4.5 -13.6   0. ]
 [  2.   -3.  -11.4   0. ]]
[[  2.    0.    0.    0. ]
 [  3.    2.5   0.    0. ]
 [  1.   -4.5 -13.6   0. ]
 [  2.   -3.  -11.4  -1. ]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Back substitution

Now that you have L and U we can do some quick back substitution. The compontents of $\vec{y}$ and $\vec{x}$ can we written down fairly easily from Eqs. 6.35 and 6.36, and in fact $\vec{y}$ in the 3 element case is given explicitly. The generalization for $\vec{y}$ is straightforward; write a for loop to calculate $\vec{y}$, then use $\vec{y}$ and another for loop to calculate $\vec{x}$.

In [ ]:
# code

### Verify

Ensure that your solution is correct. It should satisfy all 4 simultaneous equations, and/or you could just do some matrix math. `np.matmul` broadcast matrices and vectors correctly.

Now that we've done things the hard way, let's also verify that `np.linalg.solve` finds the same answer.